In [1]:
from res_sfcn import ResSFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression
import matplotlib.pyplot as plt

2021-12-28 18:40:09.477804: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
name = 'res_sfcn'
index=1

batch_size = 8
gpu_num = 8
cpu_workers = 8
epochs_num = 64

In [3]:
# col_list = pd.read_csv('fields/subcortical_volumes.csv')['field id'].to_list()

# columns=['path']
# for col in col_list:
#     columns.append(str(col)+'-2.0')

In [4]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid').dropna()
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid').dropna()
test_df = pd.read_csv('csv/split_test.csv', index_col='eid').dropna()

In [5]:
input_dim = [182, 218, 182]
num_output = len(train_df.columns)-1

In [6]:
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='quantile')

scaler_instance = train_gen.get_scaler_instance()

In [7]:
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False)

In [8]:
test_gen = VolumeDataGeneratorRegression(
    sample_df=test_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False
)

In [9]:
model = ResSFCN(
        input_dim=[182, 218, 182, 1], 
        output_dim=num_output,
        conv_num_filters=[32, 64, 64, 128, 256, 256], 
        conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
        conv_strides=[1, 1, 1, 1, 1, 1],
        conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
        pooling_size=[2, 2, 2, 2, 2],
        pooling_type=['max_pool', 'max_pool', 'max_pool', 'max_pool', 'max_pool'],
        batch_norm=True,
        dropout=False,
        softmax=False,
        res_pooling=False,
        gpu_num=gpu_num,
        name=name+'_'+str(index)
        )
model.compile(learning_rate=3e-4)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2021-12-28 18:40:21.731549: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-28 18:40:21.733536: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-28 18:40:22.142587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-28 18:40:22.145490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-28 18:40:22.148217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:07:0

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

ValueError: Operands could not be broadcast together with shapes (22, 27, 22, 64) (22, 27, 22, 32)

In [12]:
pool_shape = (3,4,5)

pool_shape = (x+1 for x in pool_shape)
pool_shape

<generator object <genexpr> at 0x7f64b42812e0>

In [ ]:
model.train_generator(train_gen, vaalid_gen, batch_size=batch_size, epochs=epochs_num, workers=cpu_workers)

In [ ]:
model.load_weights('weights/checkpoint_' + name + '_' + str(index))
model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)